In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib

matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skimage as sk
import pandas as pd
import holoviews as hv
import dask.array as da
import dask.dataframe as dd
import xarray as xr
import h5py
import pickle
import copy

from scipy import ndimage as ndi
from skimage.segmentation import watershed
from ipywidgets import (
    interact,
    interactive,
    fixed,
    interact_manual,
    FloatSlider,
    IntSlider,
    Dropdown,
    IntText,
    SelectMultiple,
    Select,
    IntRangeSlider,
    FloatRangeSlider,
)
from skimage import filters, transform
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from holoviews.operation.datashader import regrid

In [ ]:
hv.extension("bokeh")

In [ ]:
test = tr.hdf5_viewer(headpath)

In [ ]:
test.view(vmin=0, vmax=10000)

In [ ]:
import pickle as pkl


class variant_overlay(tr.hdf5_viewer):
    def __init__(
        self,
        headpath,
        trench_timepoint_df_path,
        display_values_list=[],
        persist_data=False,
        bbox_color="grey",
        bbox_alpha=0.5,
    ):
        # display_values_list is a list of strings of columns in the trench-timepoint dataframe
        # the trench-timepoint dataframe must havbe exactly 1 entry per trench per timepoint!
        super(variant_overlay, self).__init__(headpath, persist_data=persist_data)

        self.display_values_list = display_values_list
        self.bbox_color = bbox_color
        self.bbox_alpha = bbox_alpha

        self.pixel_microns = self.metadata["pixel_microns"]

        with open(headpath + "/kymograph/metadata.pkl", "rb") as handle:
            kymograph_metadata = pkl.load(handle)["kymograph_params"]

        self.trench_length = (
            kymograph_metadata["padding_y"] + kymograph_metadata["ttl_len_y"]
        )
        self.trench_width = kymograph_metadata["trench_width_x"]

        self.single_trench_timepoint_df = dd.read_parquet(
            trench_timepoint_df_path, engine="pyarrow"
        ).compute()
        self.single_trench_timepoint_df = (
            self.single_trench_timepoint_df.reset_index()
            .set_index(["fov", "timepoints"])
            .sort_index()
        )

    def get_extents(self, y_coords, x_coords):

        left = x_coords - (self.trench_width // 2)
        bottom = y_coords + self.trench_length
        right = x_coords + (self.trench_width // 2)
        top = y_coords
        extents_arr = np.stack([left, bottom, right, top])

        return extents_arr

    def get_trench_rectangles(self, trench_timepoint_df):

        y_coord = (trench_timepoint_df["y (local)"] / self.pixel_microns).astype(int)
        x_coord = (trench_timepoint_df["x (local)"] / self.pixel_microns).astype(int)

        extents_arr = self.get_extents(y_coord, x_coord)

        return extents_arr

    def make_rectangle_img(self, fov, channel, timepoint):
        ##open some dataframe and process; make sure everything relevent is in memory for speed (should be minimal anyways)
        ##possibly write all of this as a subclass or something

        selected_df = self.single_trench_timepoint_df.loc[fov, timepoint]

        extents_arr = self.get_trench_rectangles(selected_df)
        selected_df["Left"] = extents_arr[0]
        selected_df["Bottom"] = extents_arr[1]
        selected_df["Right"] = extents_arr[2]
        selected_df["Top"] = extents_arr[3]

        rectangles = hv.Rectangles(
            data=selected_df,
            kdims=["Left", "Bottom", "Right", "Top"],
            vdims=self.display_values_list,
        )
        rectangles = rectangles.opts(color=self.bbox_color, alpha=self.bbox_alpha)

        return rectangles

    def view_overlay(
        self, width=1000, height=1000, cmap="Greys_r", vmin=None, vmax=None
    ):
        main_view = self.view(
            width=width, height=height, cmap=cmap, vmin=vmin, vmax=vmax
        )
        main_view = main_view.opts(tools=[])
        dims = main_view.dimensions()

        dmap = hv.DynamicMap(self.make_rectangle_img, kdims=dims)
        dmap = dmap.opts(tools=["hover"])

        overlay = main_view * dmap

        return overlay

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/GFP/"

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/GFP/"

overlay_handle = tr.variant_overlay(
    headpath,
    "/home/de64/scratch/de64/sync_folder/2021-09-17_snakemake_lDE15_V2/2021-11-17_lDE15_Analysis_Trench-Timepoint",
    display_values_list=["gfp/mchy Ratio", "dark_gfp"],
    persist_data=False,
)

In [ ]:
overlay_handle.view_overlay(vmin=0, vmax=10000)

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/GFP/"

interactive_kymograph = tr.kymograph_interactive(headpath)
viewer = tr.hdf5_viewer(headpath, persist_data=False)

In [ ]:
test_view = viewer.view(width=1200)
test_view = test_view.opts(tools=[])

In [ ]:
type(test_view)

In [ ]:
dims = test_view.dimensions()

In [ ]:
dims

In [ ]:
dimtest = dims[0]

In [ ]:
dim_index = dimtest.values

In [ ]:
def make_rectangle_img(fov, channel, time):
    ##open some dataframe and process; make sure everything relevent is in memory for speed (should be minimal anyways)
    ##possibly write all of this as a subclass or something
    coords = [(0, 0, 100, 100)]
    return hv.Rectangles(coords)

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="04:00:00",
    local=False,
    n_workers=50,
    death_timeout=5.0,
    memory="16GB",
    working_directory="/home/de64/scratch/de64/temp/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
single_trench_timepoint_df = dd.read_parquet(
    "/home/de64/scratch/de64/sync_folder/2021-09-17_snakemake_lDE15_V2/2021-11-17_lDE15_Analysis_Trench-Timepoint",
    engine="pyarrow",
).compute()

In [ ]:
single_trench_timepoint_df = (
    single_trench_timepoint_df.reset_index()
    .set_index(["fov", "timepoints"])
    .sort_index()
)

In [ ]:
meta_handle = tr.pandas_hdf5_handler(
    "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/GFP/metadata.hdf5"
)
metadata = meta_handle.read_df("global", read_metadata=True).metadata
pixel_microns = metadata["pixel_microns"]

import pickle as pkl

with open(
    "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/GFP/kymograph/metadata.pkl",
    "rb",
) as handle:
    kymograph_metadata = pickle.load(handle)["kymograph_params"]

trench_length = kymograph_metadata["padding_y"] + kymograph_metadata["ttl_len_y"]
trench_width = kymograph_metadata["trench_width_x"]

In [ ]:
def get_extents(y_coords, x_coords, trench_length, trench_width):

    left = x_coords - (trench_width // 2)
    bottom = y_coords + trench_length
    right = x_coords + (trench_width // 2)
    top = y_coords
    extents_arr = np.stack([left, bottom, right, top])

    return extents_arr


def get_trench_rectangles(
    trench_timepoint_df, pixel_microns, trench_length, trench_width
):

    y_coord = (trench_timepoint_df["y (local)"] / pixel_microns).astype(int)
    x_coord = (trench_timepoint_df["x (local)"] / pixel_microns).astype(int)

    extents_arr = get_extents(y_coord, x_coord, trench_length, trench_width)

    return extents_arr


def make_rectangle_img(
    fov,
    channel,
    timepoint,
    trench_timepoint_df=single_trench_timepoint_df,
    pixel_microns=pixel_microns,
    trench_length=trench_length,
    trench_width=trench_width,
    alpha=0.5,
):
    ##open some dataframe and process; make sure everything relevent is in memory for speed (should be minimal anyways)
    ##possibly write all of this as a subclass or something

    selected_df = single_trench_timepoint_df.loc[fov, timepoint]

    extents_arr = get_trench_rectangles(
        selected_df, pixel_microns, trench_length, trench_width
    )
    selected_df["Left"] = extents_arr[0]
    selected_df["Bottom"] = extents_arr[1]
    selected_df["Right"] = extents_arr[2]
    selected_df["Top"] = extents_arr[3]

    rectangles = hv.Rectangles(
        data=selected_df,
        kdims=["Left", "Bottom", "Right", "Top"],
        vdims=["gfp/mchy Ratio"],
    )
    rectangles = rectangles.opts(alpha=0.5)

    return rectangles

In [ ]:
test = hv.Rectangles(
    data=single_trench_timepoint_df.loc[1, 1],
    kdims=["Left", "Bottom", "Right", "Top"],
    vdims=["gfp/mchy Ratio"],
)

In [ ]:
test

In [ ]:
rectangles

In [ ]:
test_img = make_rectangle_img(1, None, 1)

In [ ]:
dmap = hv.DynamicMap(make_rectangle_img, kdims=dims)
dmap = dmap.opts(tools=["hover"])

In [ ]:
test = test_view * dmap

In [ ]:
test